# 10 - LangGraph y Flujos de Trabajo

## Curso de LLMs y Aplicaciones de IA

**Duración estimada:** 2.5-3 horas

---

## Índice

1. [Introducción a LangGraph](#intro)
2. [Estados y Grafos](#estados)
3. [Flujo RAG con LangGraph](#rag)
4. [Auto-corrección](#correccion)
5. [Checkpoints y Persistencia](#checkpoints)
6. [Ejercicios prácticos](#ejercicios)

---

## Objetivos de aprendizaje

Al finalizar este notebook, serás capaz de:
- Crear grafos de estados con LangGraph
- Implementar flujos condicionales
- Añadir auto-corrección a sistemas RAG
- Usar checkpoints para persistencia

<a name="intro"></a>
## 1. Introducción a LangGraph

**LangGraph** es una librería de LangChain para crear flujos de trabajo como grafos de estados.

### ¿Por qué LangGraph?

- **Control explícito**: Define exactamente el flujo
- **Condicionales**: Diferentes caminos según resultados
- **Ciclos**: Permite iteraciones y re-intentos
- **Estado**: Mantiene información entre nodos
- **Persistencia**: Checkpoints para recuperación

In [1]:
# Install
#!pip install -q langchain langchain-groq langgraph langchain-huggingface faiss-cpu

In [2]:
import os
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

if 'GROQ_API_KEY' not in os.environ:
    os.environ['GROQ_API_KEY'] = getpass("GROQ API Key: ")

from langchain_groq import ChatGroq
llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0)
print("Configurado ✓")

GROQ API Key:  ········


Configurado ✓


<a name="estados"></a>
## 2. Estados y Grafos

En LangGraph, definimos:
- **State**: Datos que fluyen por el grafo
- **Nodes**: Funciones que procesan el estado
- **Edges**: Conexiones entre nodos

In [3]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, START, END

# Define state
class SimpleState(TypedDict):
    messages: List[str]
    current_step: str

# Define nodes
def step_one(state: SimpleState) -> SimpleState:
    messages = state["messages"] + ["Paso 1 completado"]
    return {"messages": messages, "current_step": "one"}

def step_two(state: SimpleState) -> SimpleState:
    messages = state["messages"] + ["Paso 2 completado"]
    return {"messages": messages, "current_step": "two"}

def step_three(state: SimpleState) -> SimpleState:
    messages = state["messages"] + ["Paso 3 completado"]
    return {"messages": messages, "current_step": "three"}

# Build graph
workflow = StateGraph(SimpleState)
workflow.add_node("step_one", step_one)
workflow.add_node("step_two", step_two)
workflow.add_node("step_three", step_three)

# Add edges
workflow.add_edge(START, "step_one")
workflow.add_edge("step_one", "step_two")
workflow.add_edge("step_two", "step_three")
workflow.add_edge("step_three", END)

# Compile
app = workflow.compile()
print("Grafo compilado ✓")

Grafo compilado ✓


In [4]:
# Run the graph
result = app.invoke({"messages": ["Inicio"], "current_step": ""})

print("Resultado:")
for msg in result["messages"]:
    print(f"  - {msg}")

Resultado:
  - Inicio
  - Paso 1 completado
  - Paso 2 completado
  - Paso 3 completado


### Grafos con condicionales

In [5]:
from typing import Literal

class ConditionalState(TypedDict):
    query: str
    query_type: str
    response: str

def classify_query(state: ConditionalState) -> ConditionalState:
    """Classify the query type."""
    query = state["query"].lower()
    if "precio" in query or "costo" in query:
        return {**state, "query_type": "pricing"}
    elif "horario" in query or "hora" in query:
        return {**state, "query_type": "schedule"}
    else:
        return {**state, "query_type": "general"}

def handle_pricing(state: ConditionalState) -> ConditionalState:
    return {**state, "response": "Los precios son: Básico 99€, Pro 299€, Enterprise consultar."}

def handle_schedule(state: ConditionalState) -> ConditionalState:
    return {**state, "response": "Horario: Lunes a Viernes, 9:00 a 18:00."}

def handle_general(state: ConditionalState) -> ConditionalState:
    return {**state, "response": "Para más información, contacta con soporte@empresa.com"}

def route_query(state: ConditionalState) -> Literal["pricing", "schedule", "general"]:
    return state["query_type"]

# Build conditional graph
cond_workflow = StateGraph(ConditionalState)
cond_workflow.add_node("classify", classify_query)
cond_workflow.add_node("pricing", handle_pricing)
cond_workflow.add_node("schedule", handle_schedule)
cond_workflow.add_node("general", handle_general)

cond_workflow.add_edge(START, "classify")
cond_workflow.add_conditional_edges(
    "classify",
    route_query,
    {"pricing": "pricing", "schedule": "schedule", "general": "general"}
)
cond_workflow.add_edge("pricing", END)
cond_workflow.add_edge("schedule", END)
cond_workflow.add_edge("general", END)

cond_app = cond_workflow.compile()
print("Grafo condicional compilado ✓")

Grafo condicional compilado ✓


In [6]:
# Test conditional routing
queries = [
    "¿Cuál es el precio del plan básico?",
    "¿Cuál es el horario de atención?",
    "¿Tienen servicio en México?"
]

for q in queries:
    result = cond_app.invoke({"query": q, "query_type": "", "response": ""})
    print(f"Q: {q}")
    print(f"A: {result['response']}\n")

Q: ¿Cuál es el precio del plan básico?
A: Los precios son: Básico 99€, Pro 299€, Enterprise consultar.

Q: ¿Cuál es el horario de atención?
A: Horario: Lunes a Viernes, 9:00 a 18:00.

Q: ¿Tienen servicio en México?
A: Para más información, contacta con soporte@empresa.com



<a name="rag"></a>
## 3. Flujo RAG con LangGraph

In [8]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage, AIMessage

# Create vector store
docs = [
    Document(page_content="El IBI se paga anualmente basado en el valor catastral."),
    Document(page_content="El IVTM grava la titularidad de vehículos matriculados."),
    Document(page_content="El ICIO se liquida al finalizar construcciones u obras."),
    Document(page_content="Las bonificaciones pueden reducir hasta un 90% el impuesto."),
]

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

print("Vector store creado ✓")

Loading weights: 100%|█████████████████████████████████████████████████████████████████████| 103/103 [00:00<00:00, 861.33it/s, Materializing param=pooler.dense.weight]
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Vector store creado ✓


In [9]:
from typing import List
from langchain_core.messages import BaseMessage

class RAGState(TypedDict):
    messages: List[BaseMessage]
    context: str
    response: str

def retrieve_context(state: RAGState) -> RAGState:
    """Retrieve relevant documents."""
    query = state["messages"][-1].content
    docs = retriever.invoke(query)
    context = "\n".join([d.page_content for d in docs])
    return {**state, "context": context}

def generate_response(state: RAGState) -> RAGState:
    """Generate response using LLM."""
    query = state["messages"][-1].content
    context = state["context"]
    
    prompt = f"""Responde basándote en el contexto.
    
Contexto: {context}

Pregunta: {query}

Respuesta:"""
    
    response = llm.invoke(prompt)
    return {**state, "response": response.content}

# Build RAG graph
rag_workflow = StateGraph(RAGState)
rag_workflow.add_node("retrieve", retrieve_context)
rag_workflow.add_node("generate", generate_response)

rag_workflow.add_edge(START, "retrieve")
rag_workflow.add_edge("retrieve", "generate")
rag_workflow.add_edge("generate", END)

rag_app = rag_workflow.compile()
print("RAG graph compilado ✓")

RAG graph compilado ✓


In [10]:
# Test RAG
result = rag_app.invoke({
    "messages": [HumanMessage(content="¿Qué es el IBI?")],
    "context": "",
    "response": ""
})

print(f"Respuesta: {result['response']}")

Respuesta: El IBI (Impuesto sobre Bienes Inmuebles) es un impuesto que se paga anualmente y está basado en el valor catastral de un inmueble.


<a name="correccion"></a>
## 4. Auto-corrección

Añadimos un paso de verificación y corrección.

In [11]:
class CorrectionState(TypedDict):
    query: str
    context: str
    response: str
    corrected_response: str
    needs_correction: bool

def retrieve(state: CorrectionState) -> CorrectionState:
    docs = retriever.invoke(state["query"])
    context = "\n".join([d.page_content for d in docs])
    return {**state, "context": context}

def generate(state: CorrectionState) -> CorrectionState:
    prompt = f"Contexto: {state['context']}\nPregunta: {state['query']}\nRespuesta:"
    response = llm.invoke(prompt)
    return {**state, "response": response.content}

def check_response(state: CorrectionState) -> CorrectionState:
    """Check if response needs correction."""
    check_prompt = f"""¿La siguiente respuesta está basada en el contexto?
    
Contexto: {state['context']}
Respuesta: {state['response']}

Responde solo 'SI' o 'NO'."""
    
    check = llm.invoke(check_prompt)
    needs_correction = "NO" in check.content.upper()
    return {**state, "needs_correction": needs_correction}

def correct_response(state: CorrectionState) -> CorrectionState:
    """Correct the response."""
    correct_prompt = f"""Mejora esta respuesta basándote solo en el contexto.
    
Contexto: {state['context']}
Respuesta original: {state['response']}

Respuesta mejorada:"""
    
    corrected = llm.invoke(correct_prompt)
    return {**state, "corrected_response": corrected.content}

def route_correction(state: CorrectionState) -> Literal["correct", "end"]:
    return "correct" if state["needs_correction"] else "end"

# Build correction graph
corr_workflow = StateGraph(CorrectionState)
corr_workflow.add_node("retrieve", retrieve)
corr_workflow.add_node("generate", generate)
corr_workflow.add_node("check", check_response)
corr_workflow.add_node("correct", correct_response)

corr_workflow.add_edge(START, "retrieve")
corr_workflow.add_edge("retrieve", "generate")
corr_workflow.add_edge("generate", "check")
corr_workflow.add_conditional_edges("check", route_correction, {"correct": "correct", "end": END})
corr_workflow.add_edge("correct", END)

corr_app = corr_workflow.compile()
print("Grafo con corrección compilado ✓")

Grafo con corrección compilado ✓


In [12]:
# Test
result = corr_app.invoke({
    "query": "¿Cuándo se paga el IVTM?",
    "context": "",
    "response": "",
    "corrected_response": "",
    "needs_correction": False
})

print(f"Respuesta original: {result['response']}")
print(f"Necesitó corrección: {result['needs_correction']}")
if result['corrected_response']:
    print(f"Respuesta corregida: {result['corrected_response']}")

Respuesta original: El IVTM se paga anualmente, al igual que el IBI, pero se basa en la titularidad de vehículos matriculados en lugar del valor catastral de una propiedad. Por lo general, el plazo para el pago del IVTM varía según la comunidad autónoma o la región en la que se encuentra el vehículo, pero suele ser anual.
Necesitó corrección: False


<a name="checkpoints"></a>
## 5. Checkpoints y Persistencia

In [13]:
from langgraph.checkpoint.memory import MemorySaver

# Create checkpointer
memory = MemorySaver()

# Compile with checkpointer
rag_with_memory = rag_workflow.compile(checkpointer=memory)

# Run with thread_id for session tracking
config = {"configurable": {"thread_id": "session1"}}

result = rag_with_memory.invoke({
    "messages": [HumanMessage(content="¿Qué impuestos hay?")],
    "context": "",
    "response": ""
}, config=config)

print(f"Respuesta: {result['response']}")

Respuesta: Hay varios impuestos, pero en este contexto, se mencionan específicamente el impuesto que se puede reducir con bonificaciones (hasta un 90%) y el Impuesto sobre Bienes Inmuebles (IBI), que se paga anualmente basado en el valor catastral.


<a name="ejercicios"></a>
## 6. Ejercicios Prácticos

### Ejercicio: Crear un flujo con múltiples pasos

In [14]:
# Exercise: Create a workflow that:
# 1. Receives a question
# 2. Classifies the question type
# 3. Retrieves relevant info
# 4. Generates response
# 5. Checks quality
# 6. Corrects if needed

## Resumen

En este notebook hemos aprendido:

1. **LangGraph**: Crear flujos como grafos de estados
2. **Condicionales**: Routing basado en resultados
3. **RAG workflow**: Retrieve → Generate
4. **Auto-corrección**: Verificar y mejorar respuestas
5. **Checkpoints**: Persistencia de sesiones

En el siguiente notebook veremos **RAG Avanzado Agentic** con flujos completos.

---

## Referencias

- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)

In [15]:
import session_info
session_info.show(html=False)

-----
ipykernel                   7.2.0
langchain_community         0.4.1
langchain_core              1.2.9
langchain_groq              1.1.2
langchain_huggingface       NA
langgraph                   NA
session_info                v1.0.1
-----
IPython             9.10.0
jupyter_client      8.8.0
jupyter_core        5.9.1
-----
Python 3.13.1 (tags/v3.13.1:0671451, Dec  3 2024, 19:06:28) [MSC v.1942 64 bit (AMD64)]
Windows-11-10.0.26200-SP0
-----
Session information updated at 2026-02-10 07:07
